# Multimodal RAG

- Author: [Mark](https://github.com/obov)
- Design:
- Peer Review :
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

## Overview

This tutorial demonstrates how to build an **Onboarding Helper** using **LangChain**, designed to centralize and leverage **Notion-based documentation** for new employees. By integrating structured data from Notion pages, databases, and wikis into a **Retrieval-Augmented Generation (RAG)** system, the solution enables seamless access to company protocols, role-specific guides, and FAQs. New hires can query this unified knowledge base in natural language to rapidly adapt to their roles.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Notion Database Setup](#notion-database-setup)
- [Langchain Only RAG](#langchain-only-rag)
- [Apply Langgraph Basic](#apply-langgraph-step1)
- [Apply Langgraph Advanced](#apply-langgraph-step2)

### References


## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**

- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.


In [71]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [72]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain-community",
        "langchain-openai",
        "langchain-chroma",
        "langchain-core",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [73]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        # "LANGCHAIN_API_KEY": "",
        # "LANGCHAIN_TRACING_V2": "true",
        # "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        # "LANGCHAIN_PROJECT": "07-Agent",
        # "UPSTAGE_API_KEY": "",
    }
)

Environment variables have been set successfully.


In [74]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

## Notion Database Setup

We use **Notion** as our central hub for team wikis, documentation, and task management. Think of it as a flexible digital workspace that combines note-taking, databases, and collaboration tools.

Key Concepts of Notion:

- **Pages**: Individual documents (like this guide) for text, images, or embedded content.
- **Databases**: Structured tables that organize information (e.g., tasks, project trackers, SOPs) with filter/sort capabilities.

### Example Database

You can view with the **exact Notion database** used in this tutorial here: [Tutorial Example Database](https://shrouded-lantana-c42.notion.site/1870d31b38698044b3f2fdd3c2c15e4c?v=1870d31b38698086a4dd000cd1ddd37a&pvs=4)

There is a list of documents for Retrieval Augmented Generation (RAG). Every document is augmented for this tutorial. Names and contents are all virtual data.

### Setup Notion Integration

to use Notion as a knowledge base, you need to create a Notion integration.

#### 1. Get API Key

1. **Go to Notion Developers**:  
   Log in to [Notion Developers](https://developers.notion.com) → Click "View my integrations".
2. **Create a New Integration**:
   - Click "New integration".
   - Name it (e.g., MyApp Integration).
   - Select your workspace.
   - Set permissions:
     - Read content
     - Update content (if needed)
3. **Copy the API Key**:  
   After creation, copy the **Internal Integration Token**
4. **More Information**:
   - [Notion API Documentation](https://developers.notion.com/reference/intro)
   - [Notion API Key](https://developers.notion.com/docs/create-a-notion-integration)

#### 2. Find Database ID

1. **Open Notion Database**:
   Go to the database you or your team want to use → Click "Share" → "Copy link".

2. **Extract the ID**:  
   The URL looks like:  
   https://www.notion.so/your-workspace/{DATABASE_ID}?v=...  
   Copy the **32-character string** between / and ? (e.g., 1870d31b38698044b3f2fdd3c2c15e4c).


In [75]:
from langchain_community.document_loaders import NotionDBLoader

NOTION_TOKEN = "ntn" + "_L3541776489aPP4RRULRr1dAfxDeeeBoJUufhX8ON0y4tM"
DATABASE_ID = "1870d31b38698044b3f2fdd3c2c15e4c"

loader = NotionDBLoader(
    integration_token=NOTION_TOKEN,
    database_id=DATABASE_ID,
)

data = loader.load()

# If you can see list of documents, it means you successfully loaded the data from Notion.
print(len(data))

31


In [76]:
print(data[0].metadata["title"])
print(data[0].metadata["tags"])
print(data[0].page_content[:800])

On Boarding 
['Task']
New Hire Onboarding Seminar
: Held on the first Monday of each month at 2 PM in the 10F auditorium.  
Team Wikis/Notion
: Check department-specific wikis for detailed work manuals and FAQs.  
Company Notices
: [notice.xyzshop.com](https://notice.xyzshop.com/) updates daily with announcements.


## Langchain Only RAG


In [77]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

data_processed = [
    *[
        Document(
            page_content=item.page_content,
            metadata={
                # Attributes
                "title": item.metadata["title"],
                "use_title_as_page_content": False,
            },
        )
        for item in data
    ],
    *[
        Document(
            # Use title as page content for similarity search
            # If you want some documents to be retrieved more frequently, you can use this method
            page_content=item.metadata["title"],
            metadata={
                # Attributes
                "page_content": item.page_content,
                "title": item.metadata["title"],
                "use_title_as_page_content": True,
            },
        )
        for item in data
    ],
]


vector_store = Chroma.from_documents(
    documents=data_processed,
    embedding=OpenAIEmbeddings(),
)

retriever_from_notion = vector_store.as_retriever(
    search_kwargs={
        "k": 5,
    }
)

In [78]:
from langchain_core.runnables import chain
from typing import List


@chain
def context_parser(docs: List[Document]) -> str:
    return "\n\n".join(
        [
            f"# {doc.metadata['title']}\n"
            f"{doc.metadata['page_content'] if doc.metadata['use_title_as_page_content'] else doc.page_content}"
            for doc in docs
        ]
    )

In [79]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
    [
        (
            "system",
            "You are a helpful assistant for onboarding new employees. \n"
            "Please answer the question based on the following documents. \n"
            "Documents: \n"
            "{context}",
        ),
        ("human", "{question}"),
    ]
)

In [80]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [81]:
## Use LangChain Only

from langchain_core.runnables import RunnablePassthrough

langchain_only_rag_chatbot = (
    {
        "question": RunnablePassthrough(),
        "context": retriever_from_notion | context_parser,
    }
    | prompt
    | llm
)

result = langchain_only_rag_chatbot.invoke("how to use conference room?")

print(result.content)

To use a conference room at XYZ Shopping Mall, follow these steps:

1. **Login:**
   - Access the internal portal at [meet.xyzshop.com](https://meet.xyzshop.com/).
   - Log in using your `@xyzshop.com` account through Single Sign-On (SSO).
   - Approve the “Meeting Room Booking” permissions when prompted.

2. **View Meeting Rooms & Schedule:**
   - Navigate through the available rooms per floor (e.g., “10F-Alpha Room,” “10F-Beta Room,” “11F-Gamma Room,” etc.).
   - Check the availability of the rooms in a calendar view.

3. **Booking Procedure:**
   - Click on an open time slot to proceed with booking.
   - Fill out the request form with the following details:
     - Meeting Title
     - Number of Attendees
     - Required Equipment (e.g., projector, video conference tools).
   - Once booked, a Google Calendar invite will be automatically sent to all participants.

4. **Cancellation/Changes:**
   - If you need to cancel or change the reservation, click “Cancel” on the reservation detai

## Apply Langgraph Basic


In [82]:
# Apply LangGraph to retriever_from_notion

from langgraph.graph import StateGraph
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict, List
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from functools import reduce

prompt_relevance_check = ChatPromptTemplate(
    [
        "Please determine whether the following question is relevant to the retrieved document.\n"
        "If it is relevant, output 'yes'; otherwise, output 'no' only.\n"
        "Question: {question}\n"
        "Retrieved Document:\n"
        "{context}"
    ]
)


class RetrievalState(TypedDict):
    question: str
    retrieved_docs: List[Document]
    relevant_docs: List[Document]


# retriever_from_notion
def retrieve_node(state: RetrievalState) -> RetrievalState:
    question = state["question"]
    return {
        "question": question,
        "retrieved_docs": retriever_from_notion.invoke(question),
        "relevant_docs": [],
    }


def filter_relevant_docs_node(state: RetrievalState) -> RetrievalState:
    question = state["question"]
    docs = state["retrieved_docs"]
    if not docs:
        return {
            "question": question,
            "retrieved_docs": docs,
            "relevant_docs": [],
        }
    idxed_docs = reduce(
        lambda acc, item: {**acc, item[0]: item[1]},
        enumerate(docs),
        {},
    )

    is_each_docs_relevant_chain = RunnableParallel(
        # Dynamically create a chain as documents retrieved
        {
            str(idx): {
                "question": RunnablePassthrough(),
                "context": RunnableLambda(
                    lambda _, doc=doc: context_parser.invoke([doc])
                ),
            }
            | prompt_relevance_check
            | llm
            | StrOutputParser()
            for idx, doc in idxed_docs.items()
        }
    ) | RunnableLambda(lambda result: list(result.values()))

    relevance_response = is_each_docs_relevant_chain.invoke(question)
    print(relevance_response)  # ['yes', 'yes', 'no']

    return {
        "question": question,
        "retrieved_docs": docs,
        "relevant_docs": [
            doc for doc, flag in zip(docs, relevance_response) if flag == "yes"
        ],
    }


graph = StateGraph(state_schema=RetrievalState)

graph.add_node("retrieve", retrieve_node)
graph.add_node("filter_relevant_docs", filter_relevant_docs_node)

graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "filter_relevant_docs")

langgraph_retriever = graph.compile()

In [83]:
langgraph_retriever_result = langgraph_retriever.invoke(
    {"question": "how to use conference room?"}
)
print(langgraph_retriever_result["question"])
print(len(langgraph_retriever_result["retrieved_docs"]))
print(len(langgraph_retriever_result["relevant_docs"]))

['yes', 'yes', 'yes', 'yes', 'yes']
how to use conference room?
5
5


In [84]:
langgraph_applied_rag = (
    {
        "question": RunnablePassthrough(),
        # LangGraph applied to retriever_from_notion
        # before: "context": retriever_from_notion | context_parser,
        "context": {
            "question": RunnablePassthrough(),
        }
        | langgraph_retriever
        | RunnableLambda(lambda result: result["relevant_docs"])
        | context_parser,
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [85]:
result = langgraph_applied_rag.invoke("how to use conference room?")

print(result)

['yes', 'yes', 'yes', 'yes', 'yes']
To use a conference room at XYZ Shopping Mall, follow these steps:

1. **Login**: Access the internal portal at [meet.xyzshop.com](https://meet.xyzshop.com/) using Single Sign-On (SSO) with your `@xyzshop.com` account. Approve any prompts for “Meeting Room Booking” permissions.

2. **View Meeting Rooms & Schedule**: Browse the available conference rooms, which are listed per floor (e.g., “10F-Alpha Room,” “10F-Beta Room,” “11F-Gamma Room,” etc.). Check the availability of rooms in the calendar view and click on an open time slot to start the booking process.

3. **Booking Procedure**:
   - Fill out the booking request form with the following details:
     - Meeting Title
     - Number of Attendees
     - Required Equipment (e.g., projector, video conference tools)
   - After successfully booking the room, a Google Calendar invite will automatically be sent to all participants.

4. **Handling Conflicts**: If there is a scheduling conflict, the system 

## Apply Langgraph Advanced


In [96]:
from langgraph.graph import StateGraph
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from typing import TypedDict, List
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

prompt_split_question = ChatPromptTemplate(
    [
        "You are an assistant that helps refine and decompose complex questions.\n"
        "Your task is to split the given question into a few concise sub-questions **only if necessary**.\n"
        "Do not introduce any new topics or unrelated details.\n"
        "Keep the sub-questions **directly relevant to the original question**.\n"
        "If the question is already specific, return it as is.\n"
        "Ensure that no extra interpretations or additional information beyond the provided question are included.\n"
        "\n"
        "Original Question: {question}\n"
        "Output (one or more refined sub-questions, separated by newlines):"
    ]
)


class QuestionState(TypedDict):
    question: str
    sub_questions: List[str]


# Node to split question
def split_question_node(state: QuestionState) -> QuestionState:
    question = state["question"]
    response = (
        prompt_split_question
        | llm
        | StrOutputParser()
        | RunnableLambda(lambda result: result.replace("\n\n", "\n"))
    ).invoke({"question": question})

    # Convert response to list
    sub_questions = response.split("\n") if "\n" in response else [response]
    print("===== sub_questions =====")
    print(sub_questions)
    return {
        "question": question,
        "sub_questions": sub_questions,
    }


graph = StateGraph(state_schema=QuestionState)

graph.add_node("split_question", split_question_node)
graph.set_entry_point("split_question")

langgraph_question_splitter = graph.compile()

# Example executions
question = "I need to check the current inventory levels for an upcoming product launch. How can I request an inventory status report from the Operations Management Team, and what key details should I include in my request?"
result = langgraph_question_splitter.invoke({"question": question})

question = "how to use conference room?"
result = langgraph_question_splitter.invoke({"question": question})

===== sub_questions =====
['How can I request an inventory status report from the Operations Management Team?  ', 'What key details should I include in my request for the inventory status report?']
===== sub_questions =====
['- What are the steps to book a conference room?', '- What equipment is available in the conference room?', '- What are the rules or guidelines for using the conference room?']


In [98]:
def list_to_dict(l):
    return {str(i): v for i, v in enumerate(l)}

In [99]:
def dict_to_dynamic_runnable(runnable):
    # Convert dictionary to RunnableParallel Dynamically
    @chain
    def _dic_to_runnable(d):
        return RunnableParallel(
            {k: (RunnableLambda(lambda x, key=k: x[key]) | runnable) for k in d.keys()}
        ).invoke(d)

    return _dic_to_runnable

In [100]:
prompt_summarize_sub_answers = ChatPromptTemplate(
    [
        (
            "system",
            "You are an assistant summarizing multiple responses for better readability.\n"
            "Please consolidate the following sub answers into a clear and concise response.\n"
            "Ensure the final answer is not too long while maintaining the key points.\n"
            "Sub Answers: {sub_answers}",
        ),
        ("human", "My question was {question}. Summarize the key points clearly."),
    ]
)

In [101]:
sub_answers_chain = (
    {
        "question": RunnablePassthrough(),
    }
    | langgraph_question_splitter
    | RunnableLambda(lambda result: list_to_dict(result["sub_questions"]))
    | dict_to_dynamic_runnable(langgraph_applied_rag)
    | RunnableLambda(lambda result: list(result.values()))
)

In [104]:
chat_bot = (
    {
        "question": RunnablePassthrough(),
        "sub_answers": sub_answers_chain,
    }
    | prompt_summarize_sub_answers
    | llm
)

In [105]:
response = chat_bot.invoke(
    "I need to check the current inventory levels for an upcoming product launch.\n"
    "How can I request an inventory status report from the Operations Management Team,\n"
    "and what key details should I include in my request?"
)
print(response.content)

===== sub_questions =====
['How can I request an inventory status report from the Operations Management Team?  ', 'What key details should I include in my request for the inventory status report?']
['no', 'yes', 'yes', 'no', 'no']
['no', 'no', 'no', 'yes', 'yes']
To request an inventory status report for your upcoming product launch, follow these steps:

1. **Identify the Contact:** Reach out to Jiwoo Shin (Assistant Manager, jiwoo.shin@xyzshop.com), with Hyeonseo Kim (Junior Staff, hyeonseo.kim@xyzshop.com) as a backup.

2. **Compose Your Email:** Include the following key details:
   - Your name and position.
   - Purpose of the request (e.g., checking inventory levels for a product launch).
   - Specific timeframe for the inventory needed (e.g., current levels).
   - Preferred data format (e.g., Excel, PDF).
   - Key metrics to include (e.g., total inventory levels, discrepancies).
   - Deadline for the report (e.g., "Please send by Friday 5:00 PM").
   - Your contact information fo